In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import h5py
import analysis_tools
from analysis_tools import get_dist_matrix, load_model, get_latent, PCA_on_latent
from sklearn.manifold import MDS

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams.update({
    'text.latex.preamble': r'\usepackage{amsfonts}'
})

ModuleNotFoundError: No module named 'infoNCE'

## Hopf Bifurcation

In [3]:
model_folder = '../cartographers'

dataname = 'hopfSDE200_tmax50_n30_D0d001.h5'
model_file = model_folder + '/cartographer.pt'


num_reads = 200

model = load_model(model_file)
stride= 4
dists, parameters = get_dist_matrix(dataname, model, num_reads, stride, fromjl=True)
#dists, parameters = analysis_tools.get_dist_matrix_fhn(dataname, model, num_reads)
latent, parameters2 = get_latent(dataname, model, num_reads, stride, fromjl=True)
latent_pca, latent_explained_variance, latent_pca_transform = PCA_on_latent(latent)

/Users/nromeo/MIT Dropbox/Nicolas Romeo/Chicago/Jerison/DynCarto/analysis/analysis_tools.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(filename, map

ModuleNotFoundError: No module named 'infoNCE'

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(latent_pca[:num_reads,0], latent_pca[:num_reads,1],s= 40, c=np.array([p[0,0] for p in parameters]), cmap=plt.cm.bwr, edgecolor="k", alpha=0.8)

plt.gca().set_box_aspect(1)
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
#plt.savefig("./figures/Hopf.pdf")

## SNIC-Hopf system

In [ ]:
#datanamefhn = 'ExamplesFhN_D0d01_t500.h5'
#dataname = 'SNIChopfSDE1024_tmax50_n30_D0d0005.h5'
# generate the data using SDEsnicHopf.jl!

model_folder = '../cartographers'

model_file = model_folder + '/cartographer.pt'


num_reads = 1024

model = load_model(model_file)

stride= 4 # downsampling in time of trajectories
latent, parameters = get_latent(dataname, model, num_reads, stride, fromjl=True)

latent_pca, latent_explained_variance, latent_pca_transform = PCA_on_latent(latent)

In [ ]:
## generate phase labels:
def classify_snic_hopf(k,a):
    if k<0:
        if k**2+a**2 < 1:
            return 1 # ordered
        else:
            return 0 # disordered
    else:
        if np.abs(a) > 1:
            return 2 # oscillating
        return 1 # ordered

labels = np.array([classify_snic_hopf(p[0,0],(p[0,1]-p[1,0])/2) for p in parameters])

In [ ]:
plt.figure(figsize=(5,5))


plt.scatter(latent_pca[:num_reads,0], latent_pca[:num_reads,1],s= 40, c=labels, cmap=plt.cm.viridis, edgecolor="k", alpha=0.8)

plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.gca().set_box_aspect(1)

In [ ]:
from sklearn.cluster import KMeans

# K means clustering in latent space. 
kmeans = KMeans(n_clusters=3, random_state=10, n_init="auto").fit(latent)
kmeans.labels_

In [ ]:
plt.figure(figsize=(5,5))

plt.scatter(latent_pca[:num_reads,0], latent_pca[:num_reads,1],s= 40, c=kmeans.labels_, cmap=plt.cm.inferno, edgecolor="k", alpha=0.8)

plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.gca().set_box_aspect(1)

In [ ]:
plt.figure()
param_k = np.array([p[0,0] for p in parameters])
param_a = np.array([(p[0,1]-p[1,0])/2 for p in parameters])
plt.subplot(1,2,1)
plt.scatter(param_k, param_a, s=20, c=np.array(labels), cmap=plt.cm.viridis, edgecolor='k', alpha=1)

plt.xlabel("k")
plt.ylabel("a")
plt.gca().set_box_aspect(1)

plt.subplot(122)
plt.scatter(param_k, param_a, s=20, c=kmeans.labels_, cmap=plt.cm.inferno, edgecolor='k', alpha=1)
plt.xlabel("k")
plt.ylabel("a")
plt.gca().set_box_aspect(1)
plt.tight_layout()
plt.show()
